In [2]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [3]:
mlflow.dspy.autolog()

/Users/aidand/dev/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import dspy
import os
from dotenv import load_dotenv
from IPython.display import Image, display

load_dotenv()

lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

In [5]:
response = lm("What is the capital of France?")
print(response)

['The capital of France is Paris.']


Trace(request_id=1c6987fd58bb4a2381127fae95d64a82)

In [6]:
qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")
print(response.response)

lm.inspect_history(n=1)

In Linux, "high memory" and "low memory" refer to different regions of the system's memory address space, particularly in the context of how the kernel manages memory for processes.

- **Low Memory**: This typically refers to the memory that is directly accessible by the kernel and can be used by processes without any special handling. In a 32-bit system, this is usually the first 896 MB of RAM (though this can vary based on the architecture and configuration). Low memory is used for kernel data structures and for user processes that require direct access to memory.

- **High Memory**: This refers to memory that is above the addressable range of the kernel in a 32-bit system. Since the kernel can only directly access a limited amount of memory (due to the 4 GB address space limitation), any memory above this limit is considered high memory. Processes can still use this memory, but the kernel must use special mechanisms (like paging) to access it. High memory is more relevant in systems

Trace(request_id=550a889132af46c9b8ca417da08c2b0c)

In [7]:
cot = dspy.ChainOfThought('question -> response')
cot(question="should curly braces appear on their own line?")

Prediction(
    reasoning='The placement of curly braces on their own line is largely a matter of coding style and conventions. In some programming languages and style guides, such as the Allman style, it is common to place opening curly braces on a new line to enhance readability. This can help visually separate blocks of code. In contrast, other styles, like K&R style, place the opening brace on the same line as the control statement, which can save vertical space. Ultimately, whether curly braces should appear on their own line depends on the coding standards being followed in a particular project or organization.',
    response="Curly braces can appear on their own line depending on the coding style you are following. If you are using a style that emphasizes readability, such as the Allman style, then yes, they should be on their own line. If you are following a more compact style like K&R, then they would typically be on the same line. It's best to adhere to the conventions of the

Trace(request_id=010db365d4a746a19253e1e58fdbb57d)

In [8]:
import ujson
from dspy.utils import download

# Download question--answer pairs from the RAG-QA Arena "Tech" dataset.
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl")

with open("ragqa_arena_tech_examples.jsonl") as f:
    data = [ujson.loads(line) for line in f]

In [9]:
# Inspect one datapoint.
data[0]

{'question': 'why igp is used in mpls?',
 'response': "An IGP exchanges routing prefixes between gateways/routers.  \nWithout a routing protocol, you'd have to configure each route on every router and you'd have no dynamic updates when routes change because of link failures. \nFuthermore, within an MPLS network, an IGP is vital for advertising the internal topology and ensuring connectivity for MP-BGP inside the network.",
 'gold_doc_ids': [2822, 2823]}

In [10]:
data = [dspy.Example(**d).with_inputs('question') for d in data]

# Let's pick an `example` here from the data.
example = data[2]
example

Example({'question': 'why are my text messages coming up as maybe?', 'response': 'This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". \n\nHowever, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.', 'gold_doc_ids': [3956, 3957, 8034]}) (input_keys={'question'})

In [11]:
import random

random.Random(0).shuffle(data)
trainset, devset, testset = data[:200], data[200:500], data[500:1000]

len(trainset), len(devset), len(testset)

(200, 300, 500)

In [12]:
from dspy.evaluate import SemanticF1

# Instantiate the metric.
metric = SemanticF1(decompositional=True)

# Produce a prediction from our `cot` module, using the `example` above as input.
pred = cot(**example.inputs())

# Compute the metric score for the prediction.
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Question: 	 why are my text messages coming up as maybe?

Gold Response: 	 This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". 

However, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.

Predicted Response: 	 Your text messages are showing up as "maybe" because the recipient's messaging app is unsure about the sender's identity. This can occur if you're not in their contacts or if the app is trying to filter out potential spam. To resolve this, you can ask the recipient to save your number or check their messaging app settings.

Semantic F1 Score: 0.33


[Trace(request_id=41674e0cc334491fa5804516a3af9a9f), Trace(request_id=e4489feb136c4a078a8b297f6d1cb618)]

In [13]:
# Define an evaluator that we can re-use.
evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24,
                         display_progress=True, display_table=2)

# Evaluate the Chain-of-Thought program.
evaluate(cot)

Average Metric: 123.61 / 300 (41.2%): 100%|██████████| 300/300 [02:18<00:00,  2.17it/s]

2025/03/11 21:16:49 INFO dspy.evaluate.evaluate: Average Metric: 123.61108373468522 / 300 (41.2%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,"when to use c over c++, and c++ over c?","If you are equally familiar with both C++ and C, it's advisable to...",[733],"C and C++ are both powerful programming languages, but they serve ...","Use C when you need low-level system programming, performance, and...",✔️ [0.286]
1,should images be stored in a git repository?,"One viewpoint expresses that there is no significant downside, esp...","[6253, 6254, 6275, 6278, 8215]",Storing images in a Git repository can be problematic for several ...,"No, images should generally not be stored in a Git repository due ...",✔️ [0.286]


41.2

[Trace(request_id=9c2d54a58a42401090e9c6149e2b1675), Trace(request_id=e43cdd1ad5be4aa5873ea044e6340d8c), Trace(request_id=2d5d8c8bdbb2468fafe2794fa67c7d3d), Trace(request_id=24644b17911e4f5f8608f9eb4c97d29c), Trace(request_id=d7bca27745f34aa2802bddf7ba73130c), Trace(request_id=8cd4080d6ce54dcf9bc41a8705aa63bb), Trace(request_id=3442f5512077413e9f6a0fa2c6ad4fa8), Trace(request_id=09af4dd6b1374730a787754f42775a34), Trace(request_id=6215161c12044018a00f80695305639b), Trace(request_id=d76ec63328904174a60849841c1d3ef4)]

In [14]:
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl")

In [16]:
max_characters = 6000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

with open("ragqa_arena_tech_corpus.jsonl") as f:
    corpus = [ujson.loads(line)['text'][:max_characters] for line in f]
    print(f"Loaded {len(corpus)} documents. Will encode them below.")

embedder = dspy.Embedder('openai/text-embedding-3-small', dimensions=512)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)

Loaded 28436 documents. Will encode them below.
Training a 32-byte FAISS index with 337 partitions, based on 28436 x 512-dim embeddings


In [17]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)

In [18]:
rag = RAG()
rag(question="what are high memory and low memory on linux?")

Prediction(
    reasoning="High memory and low memory in Linux refer to two distinct segments of the kernel's memory space. Low memory is the portion of memory that the kernel can access directly and is always mapped in the kernel's address space. This allows the kernel to access it without any additional steps. High memory, on the other hand, is not permanently mapped in the kernel's address space, meaning that the kernel must map it temporarily when it needs to access it. This distinction is crucial for managing memory efficiently, especially in a 32-bit architecture where the total addressable memory is limited. The kernel can access low memory directly, while high memory is used for temporary data buffers and requires specific functions (like kmap) to access.",
    response="In Linux, low memory refers to the segment of memory that the kernel can access directly and is always mapped in the kernel's address space. High memory, however, is a segment that is not permanently mapped, me

Trace(request_id=347047137f6a4af68c6b6dc8d4602ba5)

In [19]:
dspy.inspect_history()





[2025-03-11T21:40:37.919814]

System message:

Your input fields are:
1. `context` (str)
2. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `response` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `response`.


User message:

[[ ## context ## ]]
[1] «As far as I remember, High Memory is used for application space and Low Memory for the kernel. Advantage is that (user-space) applications cant access kernel-space memory.»
[2] «HIGHMEM is a range of kernels memory space, but it is NOT memory you access but its a place where you put what you want to access. A typical 32bit Linux virtual memory map is like: 0x00000000-0xbfffffff: user process 

In [20]:
evaluate(RAG())

Average Metric: 95.87 / 175 (54.8%):  58%|█████▊    | 175/300 [01:24<00:47,  2.65it/s]

2025/03/11 21:43:53 ERROR dspy.utils.parallelizer: Error for Example({'question': 'apples open letter - they cant or wont backdoor ios?', 'response': 'In certain cases involving specific hardware, it has been stated that manipulating the device\'s firmware is feasible. \nA data forensics company called Cellebrite has reportedly developed the ability to unlock and access data on various iPhone models, hinting at the existence of a vulnerability they can exploit. \nThe notion that this manipulation may be actionable has some support, indicated by Apple\'s own implications in their statements, although they acknowledge the security risks of fulfilling such requests. \nThere is suspicion over whether Apple will comply with creating a system that bypasses security features due to potential risks and their apparent objection to such demands. \nApple has a financial motive in portraying the iPhone as unhackable even by the FBI, and may also be attempting to capitalize on anti-government views

Average Metric: 95.87 / 175 (54.8%):  59%|█████▊    | 176/300 [01:26<02:00,  1.03it/s]

2025/03/11 21:43:53 ERROR dspy.utils.parallelizer: Error for Example({'question': 'why isnt udp with reliability (implemented at application layer) a substitute of tcp?', 'response': "Both protocols are designed with certain guarantees and raw TCP has more guarantees than raw UDP.  \nUDP with reliability can indeed be a substitute for TCP. \nHowever, in general, TCP is pretty optimal so it's not worth going to all the extra work and adding a (large) chance of adding bugs and security flaws.", 'gold_doc_ids': [3453, 3455, 3457]}) (input_keys={'question'}): litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-iVeCJplfMaAzsC1tlwHrn5Bd on tokens per min (TPM): Limit 200000, Used 199304, Requested 2395. Please try again in 509ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. Set `provide_traceback=True

Average Metric: 96.12 / 176 (54.6%):  59%|█████▉    | 178/300 [01:27<01:23,  1.47it/s]

...

2025/03/11 21:43:57 ERROR dspy.utils.parallelizer: Error for Example({'question': 'how to search for android apps without ads?', 'response': 'You could consider using Yalp or Aurora, a fork of Yalp available on F-droid, which offers multiple search filters to enhance your experience. \nThe Playsearch website also offers you the option to search for apps with specific criteria without the need to install anything. \nAppFilter, a new website currently in beta, offers a platform to search for apps applying filters for numerous features and is free to use.', 'gold_doc_ids': [658, 659, 661]}) (input_keys={'question'}): litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-iVeCJplfMaAzsC1tlwHrn5Bd on tokens per min (TPM): Limit 200000, Used 199180, Requested 3151. Please try again in 699ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 

Average Metric: 97.12 / 178 (54.6%):  92%|█████████▏| 277/300 [01:30<00:07,  3.06it/s]

2025/03/11 21:43:57 WARNING dspy.utils.parallelizer: Execution cancelled due to errors or interruption.


Exception: Execution cancelled due to errors or interruption.

[Trace(request_id=7abcaa57cc2741bfa36e516c17f0c3dd), Trace(request_id=879055201af14a4a80b6753edf81d2f6), Trace(request_id=d6ebbbb2034d41d29f961756a670eb91), Trace(request_id=02b13a2dff724b07b21aca927fda456b), Trace(request_id=5a6d4938cd8d4a2ab56840c29f2412d1), Trace(request_id=e3683724111d4cd2b0c43fb32ff13f0d), Trace(request_id=2c321709c794473c92a5e23112d3e45c), Trace(request_id=3b1db06a35a14969add0458d9b0dc3a1), Trace(request_id=1327755fde9b447a855a717f6af6aa5b), Trace(request_id=0c5174c3d69f4145b7ff1dc8e2f44459)]

In [21]:
tp = dspy.MIPROv2(metric=metric, auto="medium", num_threads=24)  # use fewer threads if your rate limit is small

optimized_rag = tp.compile(RAG(), trainset=trainset,
                           max_bootstrapped_demos=2, max_labeled_demos=2,
                           requires_permission_to_run=False)

2025/03/11 21:45:50 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 160

2025/03/11 21:45:50 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/11 21:45:50 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/11 21:45:50 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 12%|█▎        | 5/40 [01:01<07:07, 12.23s/it]


Bootstrapped 2 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Bootstrapping set 4/19


 15%|█▌        | 6/40 [01:03<06:01, 10.62s/it]


...


Average Metric: 16.44 / 25 (65.8%): 100%|██████████| 25/25 [00:01<00:00, 21.82it/s]

2025/03/11 22:14:19 INFO dspy.evaluate.evaluate: Average Metric: 16.4383194776683 / 25 (65.8%)
2025/03/11 22:14:19 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 65.75 on minibatch of size 25 with parameters ['Predictor 0: Instruction 7', 'Predictor 0: Few-Shot Set 18'].
2025/03/11 22:14:19 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [54.85, 62.78, 64.67, 61.87, 59.34, 58.56, 59.53, 61.09, 56.37, 63.19, 59.11, 61.01, 58.16, 57.32, 57.45, 66.52, 63.72, 60.46, 66.46, 60.73, 59.83, 57.62, 60.83, 65.75]
2025/03/11 22:14:19 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [56.13, 61.12, 61.14]
2025/03/11 22:14:19 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 61.14
2025/03/11 22:14:19 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/03/11 22:14:19 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 28 / 28 - Full Evaluation =====
2025/03/11 22:14:19 INFO dspy.teleprompt.mipro_optimizer_v2: D


Average Metric: 98.36 / 160 (61.5%): 100%|██████████| 160/160 [01:27<00:00,  1.82it/s]

2025/03/11 22:15:47 INFO dspy.evaluate.evaluate: Average Metric: 98.36471488348802 / 160 (61.5%)
2025/03/11 22:15:47 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 61.48
2025/03/11 22:15:47 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [56.13, 61.12, 61.14, 61.48]
2025/03/11 22:15:47 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 61.48
2025/03/11 22:15:47 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/03/11 22:15:47 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/03/11 22:15:47 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 61.48!


[Trace(request_id=7b442a793d72442e818af8f0d88222fc), Trace(request_id=7bc861d287274bde8d91d6de93b90809), Trace(request_id=7a4aac3d04cf40eb802403aa3ecf2a2e), Trace(request_id=607f21d4530d4cda9758f9b180c3aeb2), Trace(request_id=f38d7769514840a98873acb99e4c0272), Trace(request_id=d0a40a8e7224488da66f733fbe39c9c3), Trace(request_id=462588b1a0864b98bcb52d1e9dd551c0), Trace(request_id=21fe3f405c884b56aa0017b966948758), Trace(request_id=8e3e8099f986416284ed8b448ca11f7a), Trace(request_id=6a60d50aedab407aa1ce1cdb890ddbc9)]

In [22]:
baseline = rag(question="cmd+tab does not work on hidden or minimized windows")
print(baseline.response)

You are correct that cmd+tab does not work on hidden or minimized windows. To switch back to a minimized app, you must first switch to another application and let it take focus. This means you cannot directly cycle back to a minimized window using cmd+tab without first selecting a different app.


Trace(request_id=8e524e151a08437c84238d1cbe29d68c)

In [23]:
pred = optimized_rag(question="cmd+tab does not work on hidden or minimized windows")
print(pred.response)

The Command + Tab shortcut on macOS is designed to switch between open applications, but it does not activate or bring back minimized windows directly. When you use Command + Tab, it cycles through the applications that are currently open, but if an application is minimized, it will not be restored until you select it from the application switcher.

To manage this behavior, you can try the following:

1. **Adjust Mission Control Settings**: Go to System Preferences > Mission Control and uncheck the option "When switching to an application, switch to a Space with open windows for the application." This may help in switching applications more effectively.

2. **Use Option Key**: When you are cycling through applications with Command + Tab, you can hold the Option key while selecting an application. This allows you to switch focus to another app before returning to a minimized one.

3. **Third-Party Applications**: Consider using third-party applications like Witch, which can provide more

Trace(request_id=0f61ade5206d4bd6a8a12b2ab7728a84)

In [24]:
evaluate(optimized_rag)

Average Metric: 189.83 / 300 (63.3%): 100%|██████████| 300/300 [03:29<00:00,  1.43it/s]

2025/03/11 22:39:09 INFO dspy.evaluate.evaluate: Average Metric: 189.82949313136845 / 300 (63.3%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,"when to use c over c++, and c++ over c?","If you are equally familiar with both C++ and C, it's advisable to...",[733],The context provides insights into the strengths and weaknesses of...,You should consider using C over C++ in the following scenarios: 1...,✔️ [0.400]
1,should images be stored in a git repository?,"One viewpoint expresses that there is no significant downside, esp...","[6253, 6254, 6275, 6278, 8215]",The context discusses the pros and cons of storing images in a Git...,Storing images in a Git repository can be problematic due to Git's...,✔️ [0.500]


63.28

[Trace(request_id=c7001fee917349989ab5760ab6563db4), Trace(request_id=d821f71fcf5241a78a19f578f29b363e), Trace(request_id=f8a7eabd8dd74da7a163278e9c6310ee), Trace(request_id=7bbd4821d7f74a588e68bb9541aac793), Trace(request_id=2c86432064434d6ea53f6b7c256e6549), Trace(request_id=0f7085e0ad6b40d9ac9e47b6c6ebef0f), Trace(request_id=aaefe55fb63346829a178dbb1743bc3e), Trace(request_id=978a2be879244d5e98aad70d7df105eb), Trace(request_id=81bfe5fa24124ab4bd6e094e72c948f0), Trace(request_id=d37993de729b4d8b9c526b9b3968b49f)]

In [26]:
cost = sum([x['cost'] for x in lm.history if x['cost'] is not None])  # in USD, as calculated by LiteLLM for certain providers
cost

1.7683434000000022

In [27]:
optimized_rag.save("optimized_rag.json")

loaded_rag = RAG()
loaded_rag.load("optimized_rag.json")

loaded_rag(question="cmd+tab does not work on hidden or minimized windows")

Prediction(
    reasoning='The context discusses the behavior of the Command + Tab shortcut on macOS, particularly in relation to switching between applications and handling minimized or hidden windows. It explains that Command + Tab allows users to cycle through applications but does not activate minimized windows directly. The context also provides alternative methods and settings adjustments that can help manage application switching more effectively.',
    response='The Command + Tab shortcut on macOS is designed to switch between open applications, but it does not activate or bring back minimized windows directly. When you use Command + Tab, it cycles through the applications that are currently open, but if an application is minimized, it will not be restored until you select it from the application switcher.\n\nTo manage this behavior, you can try the following:\n\n1. **Adjust Mission Control Settings**: Go to System Preferences > Mission Control and uncheck the option "When swit

Trace(request_id=e5bc46fbdc544f0a9853f1f83b3d8b8e)